In [3]:
import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import LocalDegreeProfile
import torch.nn.functional as F
from torch_geometric.datasets import GemsecDeezer
from sklearn.metrics import accuracy_score, roc_auc_score
import pathlib

In [22]:
base_dir = pathlib.Path().resolve()
df = pd.read_csv(f'{base_dir}/datasets/fingerSimilarity.csv')
df

,DCC_ID,DCC0029,DCC0030,DCC0032,DCC0036,DCC0037,DCC0038,DCC0039,DCC0040,DCC0042,...,DCC0233,DCC0144,DCC0145,DCC0147,DCC0209,DCC0238,DCC0239,DCC0148,DCC0149,DCC0150
0,DCC0029,0.000000,0.183099,0.176471,0.338710,0.320000,0.303797,0.222222,0.218750,0.625000,...,0.450704,0.329114,0.377049,0.457831,0.189189,0.666667,0.377049,0.223684,0.614035,0.261538
1,DCC0030,0.183099,0.000000,0.428571,0.185714,0.164706,0.410959,0.629630,0.392857,0.164179,...,0.450704,0.438356,0.183099,0.406977,0.600000,0.095890,0.200000,0.576271,0.135802,0.438596
2,DCC0032,0.176471,0.428571,0.000000,0.234375,0.202532,0.375000,0.584906,0.275862,0.193548,...,0.337838,0.364865,0.230769,0.376471,0.377049,0.206349,0.269841,0.369231,0.222222,0.392857
3,DCC0036,0.338710,0.185714,0.234375,0.000000,0.289474,0.342105,0.242857,0.350877,0.283333,...,0.478261,0.368421,0.886364,0.445783,0.225352,0.410714,0.596154,0.295775,0.400000,0.227273
4,DCC0037,0.320000,0.164706,0.202532,0.289474,0.000000,0.311111,0.256098,0.192308,0.207792,...,0.325843,0.333333,0.337838,0.373737,0.157303,0.283784,0.237500,0.200000,0.289157,0.243590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,DCC0238,0.666667,0.095890,0.206349,0.410714,0.283784,0.337838,0.183099,0.213115,0.574468,...,0.414286,0.294872,0.428571,0.392857,0.135135,0.000000,0.403509,0.202703,0.600000,0.181818
302,DCC0239,0.377049,0.200000,0.269841,0.596154,0.237500,0.355263,0.222222,0.322034,0.344828,...,0.537313,0.438356,0.647059,0.440476,0.239437,0.403509,0.000000,0.328571,0.460317,0.223881
303,DCC0148,0.223684,0.576271,0.369231,0.295775,0.200000,0.513514,0.492308,0.526316,0.175676,...,0.600000,0.561644,0.273973,0.494253,0.515625,0.202703,0.328571,0.000000,0.216867,0.281690
304,DCC0149,0.614035,0.135802,0.222222,0.400000,0.289157,0.290698,0.185185,0.178082,0.622642,...,0.353659,0.378049,0.437500,0.449438,0.215190,0.600000,0.460317,0.216867,0.000000,0.267606


In [69]:
def get_top_cells(df, least_score=0.85):
    col_names = np.array(list(df.columns)[1:])
    row_names = np.array(list(df.iloc[:, 0:1].to_numpy().reshape((df.shape[0],))))
    temp = df.iloc[:, 1:]
    temp = temp[temp >= least_score]
    table = []
    for i in range(temp.shape[0]):
        for j in range(temp.shape[1]):
            score = temp.iloc[i, j]
            if not str(score) == 'nan':
                table.append([row_names[i], row_names[j], score])
    table = np.array(table)
    return table

table = get_top_cells(df, 0.85)

In [71]:
table.shape

(126, 3)